# The Quadratic Assignment Problem

In [351]:
##This is an initialization cell. Run this first
import pandas as pd
import numpy as np
from itertools import product
import time
import math
import matplotlib
import matplotlib.pyplot as plt
import random

## Repository

In [352]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [353]:
#REPOSITORY

#small sized matrices(under 10x10)
matrix_size_4 = 'tai4a.csv'
matrix_size_5 = 'tai5a.csv'
matrix_size_6 = 'tai6a.csv'
matrix_size_7 = 'tai7a.csv'
matrix_size_8 = 'tai8a.csv'
matrix_size_9 = 'tai9a.csv'

#medium sized matrices(ranging from 10x10 to 30x30)
matrix_size_10 = 'tai10a.csv'
matrix_size_11 = 'tai11a.csv'
matrix_size_12 = 'tai12a.csv'
matrix_size_15 = 'chr15a.csv' 
matrix_size_20 = 'chr20a.csv'
matrix_size_26 = 'bur26a.csv'

#large sized matrices(30x30 and bigger)
matrix_size_40 = 'tai40a.csv'
matrix_size_60 = 'tai60.csv'
matrix_size_80 = 'tai80.csv'
matrix_size_256 = 'tai256c.csv'

datamatrix = CSVtoNumpyArray(matrix_size_40) # Decide the size of problem to run in the code 
                                            # (clue: the number in the original name is the size)
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

## Preliminary functions

In [354]:
def Exhaustive_search(listofpermutations,MatrixLocat,MatrixFlow):
    """
    Input:
    MatrixLoc
    MatrixFlow
    listofpermutations
    
    Output:
    The optimal permutation
    the optimal cost
    in a tuple
    """
    start = time.time()
    matrix_length = len(MatrixLoc)
    no_of_permutations = len(listofpermutations)
    arraysol = []
    
    #generate the multiples (that function we are optimising)
    for j in range(no_of_permutations):
        perm = listofpermutations[j]
        total = 0
        for i in range(matrix_length - 1):
            total += MatrixLoc[i][i+1]*MatrixFlow[int(perm[i])][int(perm[i+1])]#this is that function that 
                                                #adds the products of different combinations of factories
        arraysol.append(total)
    
    finalcost = min(arraysol)
    finalindex = np.argmin(arraysol) #finds the optimal set of locations to factories(Which I stupidly 
                                        #called flow)
    end = time.time()
    thetime = end - start    
    return listofpermutations[finalindex],finalcost,thetime

## Wolf pack algorithm
### https://www.hindawi.com/journals/mpe/2014/465082/ Wolf Pack Algorithm for Unconstrained Global Optimization Hu-Sheng Wu and Feng-Ming Zhang

### So this algorithm describes 3 specific behaviours for wolves. I have redefined them to suit the problem at hand. Read the original journal for the step functions defined for each bhaviour. Otherwise I have kept to the spirit of the algorithm:

#### Scouting:
Here wolves do medium steps around where they were placed (reasonably randomly) to find a good minimum. Here I am using a last value swap as it is low in computation weight and a good analogy of the circular motion in the original step search

#### Calling
Here wolves who scouted successfully would call others closer to the lead wolf (the minimum). These are the big steps towards a successful minimum so I will find the initial permutation number (i.e. the main branch of the lead wolf) and will swap it with the wolf in question's leaing value:
EG: if the wolf in question is (2,4,6,1,3,5) and (6,5,4,3,2,1) is the current leading wolf, then the wolf in question will become: (6,4,2,1,3,5)
This is repeated for the next 2 positions.

#### Beseiging
Here the wolves surround the lead wolf. Here I will run the 6 permutations the bottom of the branches. This will be the smallest step. The best of these move onto the next step and the cycle repeats.


In [355]:
def replenish_herd(length,populationsize):
    """
    Input:
    length is the size of the matrix
    populationsize is the number of permutations you need
    
    Output:
    listofpermutations: list of lists
    """
    listofpermutations= []
    triallist = list(range(length))
    i = 0
    for i in range(populationsize):
        random.shuffle(triallist)
        dummy = triallist[:]
        listofpermutations.append(dummy)
    return listofpermutations

In [356]:
# Find the minimum
def find_minimum(listofpermutations):
    """
    Input:
    list of permutations: a list of lists
    
    Output:
    opt perm: list
    opt perm length: float
    """
    matrix_length = len(listofpermutations[0])
    arraysol = []
    #generate the multiples (that function we are optimising)
    for j in listofpermutations:
        total = 0
        for i in range(matrix_length - 1):
            if MatrixLoc[i][i+1] != 0 and MatrixFlow[int(j[i])][int(j[i+1])] != 0:
                total += MatrixLoc[i][i+1]*MatrixFlow[int(j[i])][int(j[i+1])]#this is that function that 
                                                #adds the products of different combinations of factories
            else:
                total = math.inf
        arraysol.append(total)
    finalcost = min(arraysol)
    finalindex = np.argmin(arraysol) #finds the optimal set of locations to factories(Which I stupidly 
                                        #called flow)
    return finalcost,finalindex,listofpermutations[finalindex]


In [357]:
def scout(listofpermutations):
    """
    Input:
    listofpermutations: list of lists
    
    Output:
    newlistofpermutations: list of lists
    """
    #All this does is swap the last two positions
    for i in listofpermutations:
        newlast = i[-2]
        oldlast = i[-1]
        i[-1] = newlast
        i[-2] = oldlast
        
    return listofpermutations

In [358]:
def call(listofpermutations,minindex):
    """
    Input:
    listofpermutations: list of lists
    minindex: int object with the position of the min list in the list
    
    Output:
    newlistofpermutations: list of lists
    """
    keybranch = listofpermutations[minindex][0]
    for i in listofpermutations:
        keyindex = i.index(keybranch)
        newfirst = i[keyindex]
        oldfirst = i[0]
        i[0] = newfirst
        i[keyindex] = oldfirst
    return listofpermutations
    

In [359]:
def ourpermutations(iterable, r=None):
    """
    Input:
    String or numbers separated by a space
    optional= the length that the permutations must be
    
    Output:
    a generator of permutations
    """
    
    pool = iterable
    n = len(pool)
    r = n if r is None else r
    for indices in product(range(n), repeat=r):
        if len(set(indices)) == r:
            yield list(pool[i] for i in indices)

In [360]:
def beseige(listofpermutations):
    #don't forget to catch duplicates before the next search
    """
    Input:
    listofpermutations: list of lists
    
    Output:
    newlistofpermutations: list of lists
    """
    newlistofpermutations = []
    for i in listofpermutations:
        ends = list(ourpermutations(i[-3:]))
        for j in ends:
            temp = i[:-3]
            temp= temp + j
            newlistofpermutations.append(temp)
    
    return newlistofpermutations

In [361]:
#rank post-beseige and return desired amount
def rankperms(listofpermutations,noofbest):
    """
    Input:
    list of permutations: a list of lists
    
    Output:
    opt perm: list
    opt perm length: float
    """
    matrix_length = len(listofpermutations[0])
    arraysol = []
    
    #generate the multiples (that function we are optimising)
    for j in listofpermutations:
        total = 0
        for i in range(matrix_length - 1):
            if MatrixLoc[i][i+1] != 0 and MatrixFlow[int(j[i])][int(j[i+1])] != 0:
                total += MatrixLoc[i][i+1]*MatrixFlow[int(j[i])][int(j[i+1])]#this is that function that 
                                                #adds the products of different combinations of factories
            else:
                total = math.inf
        arraysol.append(total) 
    finalindices = np.argsort(arraysol) #ranks the options
    
    #takes best number
    bestperms = []
    for i in range(noofbest):
        bestperms.append(listofpermutations[finalindices[i]])
    
    return bestperms


In [362]:
def wolfpack(MatrixLocat,MatrixFlow,Totaliterations,packsize,callweight,noofbest):
    """
    Input:
    MatrixLocat: input data numpy matrix
    MatrixFlow: input data numpy matrix
    Totaliterations: int of number of times we run the code (~100)
    packsize: int of number wolves each time(~100)
    callweight: int number of times we call the wolves(~3)
    noofbest: int number of values moving onto the next replenishment(1<noofbest<packsize/2)
    
    
    Output:
    optimal weight: float
    optimal permutation: list
    """
    #initial values
    listofpermutations = []
    length = len(MatrixLocat)
    

    for i in range(Totaliterations):
        #filling up the herd
        listofpermutations += replenish_herd(length,packsize)
        minimumperm = find_minimum(listofpermutations)[2]
        
        #scout
        listofscouted = scout(listofpermutations)
        listofscouted.append(minimumperm)
        minimumindex = find_minimum(listofscouted)[1]
        
        #call
        listofcalled = listofscouted
        for i in range(callweight):
            listofcalled = call(listofcalled,minimumindex)
            minimumindex = find_minimum(listofscouted)[1]
        
        #beseige
        listofbeseige = beseige(listofpermutations)
        listofpermutations = rankperms(listofbeseige,noofbest)
        
    return  find_minimum(listofpermutations)[::2]
        

In [363]:
Totaliterations = 100
packsize = 50
callweight = 3
noofbest = 50//3
wolfpack(MatrixLoc,MatrixFlow,Totaliterations,packsize,callweight,noofbest)

(72233,
 [29,
  17,
  34,
  31,
  3,
  26,
  39,
  4,
  16,
  21,
  36,
  19,
  27,
  6,
  20,
  12,
  15,
  9,
  22,
  35,
  32,
  37,
  0,
  25,
  30,
  5,
  24,
  8,
  10,
  7,
  38,
  2,
  13,
  14,
  11,
  28,
  1,
  33,
  23,
  18])

## Random random 

In [18]:
math.factorial(11)

39916800

In [239]:
50//3

16